### Test

In [25]:
# problems with qa chekcks that hae more than one feature class as parameter
# you can only select one feature class per parameter, this would be a problem if for example i want chek min length for two different feature classes

import prosuite as ps
import importlib.util
import inspect
from typing import List
import ipywidgets as widgets
from IPython.display import display
import os
import geopandas as gpd
import fiona

# Load quality conditions module
module_path = r"C:\\git\\Dira.ProSuiteSolution\\ProSuite.Shared\\py\\prosuite\\factories\\quality_conditions.py"

def load_quality_conditions_module(path):
    """Load the quality conditions module dynamically from the given path."""
    spec = importlib.util.spec_from_file_location("quality_conditions", path)
    quality_conditions = importlib.util.module_from_spec(spec)
    spec.loader.exec_module(quality_conditions)
    return quality_conditions

# Load the module and access Conditions class
quality_conditions_module = load_quality_conditions_module(module_path)
Conditions = quality_conditions_module.Conditions

class TestParameterType:
    """Helper class defining properties for each parameter."""
    def __init__(self, name, description, data_type, default_value, options=None):
        self.name = name
        self.description = description
        self.data_type = data_type
        self.default_value = default_value
        self.options = options

    def create_widget(self):
        """Creates a widget based on the parameter's type."""
        full_description = f"{self.description} (Type: {self.data_type})"
        layout = widgets.Layout(width='50%')
        style = {'description_width': 'initial'}

        if self.data_type == "double":
            return widgets.FloatText(value=self.default_value, description=full_description, layout=layout, style=style)
        elif self.data_type == "int":
            return widgets.IntText(value=self.default_value, description=full_description, layout=layout, style=style)
        elif self.data_type == "bool":
            return widgets.Checkbox(value=self.default_value, description=full_description, layout=layout, style=style)
        elif self.data_type == "str":
            return widgets.Text(value=self.default_value, description=full_description, layout=layout, style=style)
        elif self.data_type == "BaseDataset":
            return widgets.Dropdown(options=self.options or ["Select a .gdb first"], description=full_description, layout=layout, style=style)
        else:
            raise ValueError(f"Unsupported data type: {self.data_type}")

class TestDescriptor:
    """Represents a QA test and holds its parameters."""
    def __init__(self, name, constructor_index, display_name, description, test_parameter_types: List[TestParameterType]):
        self.name = name
        self.constructor_index = constructor_index
        self.display_name = display_name
        self.description = description
        self.test_parameter_types = test_parameter_types

    def create_condition(self, condition_name, parameter_values_by_name):
        """Generates condition code string with provided parameter values."""
        result = f"{self.name}({self.constructor_index})"
        values = ", ".join(f"{parameter_values_by_name[param.name]}" for param in self.test_parameter_types)
        return f"{condition_name} = {result}.parameters([{values}])"

def retrieve_datasets_from_gdb(gdb_path):
    """Retrieve dataset names from the selected geodatabase (.gdb)."""
    if not os.path.exists(gdb_path) or not gdb_path.endswith('.gdb'):
        print("Invalid .gdb path.")
        return []
    return fiona.listlayers(gdb_path)

def generate_test_gui():
    # Widget for GDB path input
    gdb_path_widget = widgets.Text(description="GDB Path:", placeholder="Enter .gdb path", layout=widgets.Layout(width='50%'))
    load_gdb_button = widgets.Button(description="Load Datasets", layout=widgets.Layout(width='20%'))
    dataset_options = []  # To store datasets from selected gdb

    # Callback to load datasets from gdb
    def load_datasets_from_gdb(b):
        nonlocal dataset_options
        gdb_path = gdb_path_widget.value
        dataset_options = retrieve_datasets_from_gdb(gdb_path)
        print(f"Loaded datasets: {dataset_options}")

    load_gdb_button.on_click(load_datasets_from_gdb)

    test_selector = widgets.Dropdown(
        options=[(test.display_name, test) for test in all_tests()],
        description="Test Type:",
        style={'description_width': 'initial'}
    )

    name_widget = widgets.Text(description="Name function:", style={'description_width': 'initial'})
    description_output = widgets.Output()
    param_widgets_output = widgets.Output()
    output_code = widgets.Output()
    generate_button = widgets.Button(description="Generate Code", style={'description_width': 'initial'})

    def update_parameters(change):
        selected_test = change['new']
        description_output.clear_output()
        param_widgets_output.clear_output()

        with description_output:
            print(selected_test.description)  # Display the test description

        with param_widgets_output:
            param_widgets = {}
            for param in selected_test.test_parameter_types:
                # Update options if parameter is BaseDataset
                if param.data_type == "BaseDataset":
                    param.options = dataset_options
                widget = param.create_widget()
                param_widgets[param.name] = widget
                display(widget)
            display(generate_button)

        def generate_code(_):
            output_code.clear_output()

            # Collecting parameter values from the generated widgets
            parameter_values = {name: widget.value for name, widget in param_widgets.items()}

            # Start the code generation
            with output_code:
                print(f"def {name_widget.value.lower()}_check(model, output_dir, envelope):")
                print("    specification = ps.Specification(")
                print(f"        name='{name_widget.value}',")
                print(f"        description='A QA check for {name_widget.value}'")
                print("    )")
                
                # Access the dataset by name
                dataset_name = parameter_values.get('feature_class')  # Assume 'feature_class' holds the dataset name
                print(f"    # Adding condition for dataset \"{dataset_name}\"")
                
                # Dynamically generate the condition with inline dataset creation, excluding 'feature_class'
                condition_code = f"ps.Conditions.{selected_test.name}(ps.Dataset(\"{dataset_name}\", model),"

                # Append the other parameters dynamically, excluding 'feature_class'
                for param in selected_test.test_parameter_types:
                    if param.name != 'feature_class':  # Skip 'feature_class'
                        param_value = parameter_values[param.name]
                        condition_code += f" {param.name}={param_value},"

                condition_code = condition_code.rstrip(',') + ")"  # Close the condition function and remove trailing comma
                print(f"    specification.add_condition({condition_code})")
                print(f"    print(f'Condition added for dataset: {{dataset_name}}')")
                
                # Run the verification
                print("    run_verification(specification, output_dir, envelope)")

                # Example call to the generated function
                print("\n# Example call")
                print(f"{name_widget.value.lower()}_check(model, output_dir, envelope)")

        generate_button.on_click(generate_code)

    test_selector.observe(update_parameters, names='value')

    display(gdb_path_widget, load_gdb_button, name_widget, test_selector, description_output, param_widgets_output, output_code)

def all_tests():
    """Retrieve all tests and their parameter types from the Conditions class."""
    tests = []

    for name, method in inspect.getmembers(Conditions, predicate=inspect.ismethod):
        try:
            signature = inspect.signature(method)
            parameters = signature.parameters
            docstring = method.__doc__ or "No description available."

            param_types = []
            for param_name, param in parameters.items():
                if param.annotation is str:
                    param_types.append(TestParameterType(param_name, f"{param_name} ", "str", ""))
                elif param.annotation is float:
                    param_types.append(TestParameterType(param_name, f"{param_name} ", "double", 0.0))
                elif param.annotation is int:
                    param_types.append(TestParameterType(param_name, f"{param_name} ", "int", 0))
                elif param.annotation is bool:
                    param_types.append(TestParameterType(param_name, f"{param_name} ", "bool", False))
                else:
                    param_types.append(TestParameterType(param_name, f"{param_name} ", "BaseDataset", None))

            tests.append(TestDescriptor(
                name=name,
                constructor_index=0,
                display_name=name.replace('_', ' ').title(),
                description=docstring,
                test_parameter_types=param_types
            ))

        except Exception as e:
            print(f"Could not inspect {name}: {e}")

    return tests

# Start the GUI for test generation
#generate_test_gui()
